In [1]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 367.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [2]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.9 MB/s eta 0:00:00


In [4]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='enter api key',
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("Simba was the  ....")
print(response.content)

...main character and the future king in Disney's 'The Lion King'.


In [6]:
# Define the updated system prompt
context = [{'role': 'system', 'content': """
You are an enthusiastic ice cream shop assistant working at
"Sundae Delights."

Your role is to warmly greet customers, take their ice cream orders, and ensure they have all the information they need.

Here are your guidelines:
- Take the customer's order, ensuring they only choose from the items on our menu, including any toppings.
- Provide a summary of their order.
- Confirm if the order is correct or if the customer wants to add anything.
- Process the payment, making sure to include the size and any toppings in the total.
- Clearly explain all available options, extras, and sizes to ensure the customer gets exactly what they want.
- Keep your responses brief and friendly.

Our Menu:
Ice Cream Flavors:
- Vanilla
- Chocolate
- Mint
- Raspberry
- Coffee

Sizes:
- Large: $4
- Medium: $3

Toppings:
- Sprinkles
- Chocolate syrup
- Crushed nuts
Each topping costs $0.75
""" }]

In [7]:
def continue_conversation(messages, model, groq_client, temperature=0):
    """
    Continue a conversation with Groq using the provided messages.

    This function sends a series of messages to Groq to get a response
    that continues the conversation based on the provided message history.

    Parameters:
    - messages (list of dict): A list of message dictionaries where each dictionary
      has a 'role' (str) and 'content' (str). 'role' should be either 'system', 'user', or 'assistant'.
    - model (str): The model to use for generating the response (e.g., "llama-3.1-70b-versatile").
    - groq_client (ChatGroq): An instance of the ChatGroq client for making API calls.
    - temperature (float, optional): Sampling temperature to use when generating responses.
      Higher values make the output more random, while lower values make it more deterministic. Default is 0.

    Returns:
    - str: The content of the response from the model.
    """
    # Create the response using Groq
    response = groq_client.invoke(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.content

In [8]:
def gradio_chat(message, history, groq_client, model):
    """
    Handles a chat interaction by appending the conversation history and the new message,
    then generates a response using Groq.

    Parameters:
    - message (str): The latest message from the user.
    - history (list of tuple): A list of tuples representing the conversation history,
      where each tuple contains (user_message, assistant_message).
    - groq_client (ChatGroq): An instance of the ChatGroq client for making API calls.
    - model (str): The model to use for generating the response (e.g., "llama-3.1-70b-versatile").

    Returns:
    - str: The content of the response from the model.
    """
    # Start with the system context instructions
    history_chat = context

    # Add the history of user and assistant messages
    for user_message, assistant_message in history:
        history_chat.append({"role": "user", "content": user_message})
        history_chat.append({"role": "assistant", "content": assistant_message})

    # Append the latest user message
    history_chat.append({"role": "user", "content": message})

    # Call Groq to get a response
    response = continue_conversation(history_chat, model=model, groq_client=groq_client, temperature=0)

    return response

In [9]:
import gradio as gr

# Define the customized Gradio textbox
InputText = gr.Textbox(
    label="Order",
    info="Enter your order details here. For example, 'I would like a large vanilla ice cream with caramel and white chocolate.'",
    scale=6
)

# Example of a Gradio interface using the customized textbox
def process_order(order):
    # Placeholder function to process the order
    return f"Your order is: {order}"

# Create the Gradio interface
iface = gr.Interface(
    fn=process_order,
    inputs=InputText,
    outputs="text"
)

# Launch the Gradio interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2a983190a5fc6c5c15.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [12]:

!pip freeze > requirements.txt

In [13]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>